In [ ]:
import numpy as np
import torch
import json


In [21]:
unk = '<UNK>'

In [22]:
def load_data(train_data, val_data, test_data):
    with open(train_data) as training_f:
        training = json.load(training_f)
    with open(val_data) as valid_f:
        validation = json.load(valid_f)
    with open(test_data) as test_f:
        test = json.load(test_f)

    tra = []
    val = []
    tes = []
    for elt in training:
        tra.append((elt["text"].split(),int(elt["stars"]-1)))
    for elt in validation:
        val.append((elt["text"].split(),int(elt["stars"]-1)))
    for elt in test:
        tes.append((elt["text"].split(),int(elt["stars"]-1)))

    return tra, val, tes

In [23]:
# Returns: 
# vocab = A set of strings corresponding to the vocabulary
def make_vocab(data):
    vocab = set()
    for document, _ in data:
        for word in document:
            vocab.add(word)
    return vocab 

In [24]:
def make_indices(vocab):
    vocab_list = sorted(vocab)  # vocab a-z 排序
    vocab_list.append(unk)      # 结尾添加unk
    word2index = {}
    index2word = {}
    for index, word in enumerate(vocab_list):
        word2index[word] = index        # word2index[word] = index
        index2word[index] = word        # index2word[index] = word
    vocab.add(unk)
    return vocab, word2index, index2word 


# Returns:
# vectorized_data = A list of pairs (vector representation of input, y)
def convert_to_vector_representation(data, word2index):
    vectorized_data = []
    for document, y in data:
        vector = torch.zeros(len(word2index)) 
        for word in document:
            index = word2index.get(word, word2index[unk])   # 查找该单词的索引。如果单词在 word2index 中不存在，则返回 <UNK> 的索引。
            vector[index] += 1
        vectorized_data.append((vector, y))
    return vectorized_data


In [25]:
print("========== Loading data ==========")
train_data, valid_data, test_data = load_data("./training.json", "./validation.json", "./test.json") # X_data is a list of pairs (document, y); y in {0,1,2,3,4}
vocab = make_vocab(train_data)      # 返回不重复的vocabuaries 集合
vocab, word2index, index2word = make_indices(vocab) # vocab 排序好a-z, <UNK>; word2index[word] => index 0, 1, 2...; index2word[index] => word "<UNK>", "hello"...


========== Loading data ==========


In [13]:
len(vocab)

65667

In [14]:
len(train_data)

8000

In [15]:
len(valid_data)

800

In [16]:
len(test_data)

800

In [12]:
print("========== Vectorizing data ==========")
train_data = convert_to_vector_representation(train_data, word2index)
valid_data = convert_to_vector_representation(valid_data, word2index)
test_data = convert_to_vector_representation(test_data, word2index)

========== Vectorizing data ==========


In [17]:
train_data[0][0].shape

torch.Size([65667])

In [18]:
valid_data[0][0].shape

torch.Size([65667])

In [19]:
test_data[0][0].shape

torch.Size([65667])

In [26]:
import json
from collections import Counter
import pandas as pd

def count_labels(data):
    # 统计每种标签的数量
    labels = [y for _, y in data]
    return Counter(labels)

# 统计每个数据集中不同标签的数量
train_counts = count_labels(train_data)
valid_counts = count_labels(valid_data)
test_counts = count_labels(test_data)

# 将统计结果转为表格
df = pd.DataFrame({
    "Train": train_counts,
    "Validation": valid_counts,
    "Test": test_counts
}).fillna(0).astype(int)

# 如果需要显示标签顺序
df = df.reindex(range(5), fill_value=0)

print(df)

   Train  Validation  Test
0   3200         320     0
1   3200         320     0
2   1600         160   160
3      0           0   320
4      0           0   320
